# Exploring Stockholm, Sweden: The battle of neighborhoods!

## Applied Data Science Capstone Project By Ekta Vats

## Table of contents:
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

Stockholm is the capital and most populous city of Sweden, and the city stretches across 14 islands where Lake Mälaren flows into the Baltic Sea. Stockholm is also the cultural, media, political, and economic centre of Sweden. Stockholm is a vibrant and exciting place, with rich cultural diversity and friendly people all around.

Want to explore the Stockholm city? Let's do some fun with data science!

In this project, we will explore the city of Stockholm, and find out the most popular places (such as museums, parks, café, restaurants, etc.), to help guide the tourists, investors and general public interested in visiting Stockholm. We would also prefer locations that are close to the city centre.

We will use the data science skills acquired as part of the IBM Data Science Professional Certification (by Coursera), and generate a few most popular neighbourhoods with information about interesting venues to visit. The details about each venue will then be suggested for the best possible exploration of the Stockholm city.

## Data <a name="data"></a>

Based on the problem definition, following factors will contribute in deciding which neighborhood is popular:
* number of museums and attractions in the neighborhood.
* number of café and restaurants in the neighborhood.
* whether the neighborhood is close to the city center.
* population density.

We will focus around the Stockholm city centre to define our neighborhoods.

Following data sources will be needed to extract the required information:

* **Wikipedia**, https://en.wikipedia.org/wiki/Stockholm_City_Centre, to obtain the information related to the districts/boroughs in the Stockholm city centre.

* **Foursquare API** to obtain the location/venue data (i.e. number of museums, attractions, café, restaurants, etc.)
* **BeautifulSoup** to parse the area information from a HTML table.
* **Geocoder** to obtain the coordinates of the Stockholm city center.
* **Folium** to visualize the maps.

### Neighborhood Candidates

So let's get started! <br>
Let's first install and import the required libraries, and then proceed with finding the latitude and longitude coordinates for the centroids of our candidate neighborhoods around the Stockholm city centre.

Import the libraries:

In [1]:
!pip install geocoder
!pip install folium

In [2]:
import pandas as pd
import requests
import numpy as np
import geocoder
import folium
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("Libraries Imported!")

Libraries Imported!


In [3]:
url = "https://en.wikipedia.org/wiki/Stockholm_City_Centre"
extracting_data = requests.get(url).text
wiki_data = BeautifulSoup(extracting_data, 'xml')
table=wiki_data.find('table')
table

<table border="1" class="wikitable sortable">
<caption>Sortable table
</caption>
<tbody><tr bgcolor="#dddddd">
<th>District</th>
<th>Area (<a href="/wiki/Hectare" title="Hectare">ha</a>)<sup class="reference" id="cite_ref-2"><a href="#cite_note-2">[2]</a></sup></th>
<th>Population<br/>(31-12-2007)</th>
<th>Density (/ha)</th>
<th><a href="/wiki/Borough" title="Borough">Borough</a></th>
<th><a href="/wiki/Province" title="Province">Province</a>
</th></tr>
<tr align="right" bgcolor="#ffdddd">
<td align="left"><a href="/wiki/Djurg%C3%A5rden" title="Djurgården">Djurgården</a></td>
<td>290</td>
<td>788</td>
<td>3</td>
<td align="left"><a class="mw-redirect" href="/wiki/%C3%96stermalm_borough" title="Östermalm borough">Östermalm</a></td>
<td align="left"><a href="/wiki/Uppland" title="Uppland">Uppland</a>
</td></tr>
<tr align="right" bgcolor="#ffdddd">
<td align="left"><a class="new" href="/w/index.php?title=Fredh%C3%A4ll&amp;action=edit&amp;redlink=1" title="Fredhäll (page does not exist)">F

In [4]:
column_names = ['District', 'Area', 'Population', 'Density', 'Borough', 'Province']
df = pd.DataFrame(columns = column_names)

# Add data
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==6:
        df.loc[len(df)] = row_data

In [5]:
print(df.shape)
df

(39, 6)


,District,Area,Population,Density,Borough,Province
0,Djurgården,290,788,3,Östermalm,Uppland
1,Fredhäll,38,"4,958",130,Kungsholmen,Uppland
2,Gustav Vasa,80,"12,911",161,Norrmalm,Uppland
3,Gärdet,459,"18,158",40,Östermalm,Uppland
4,Hedvig Eleonora,58,"10,387",179,Östermalm,Uppland
5,Hjorthagen-Värtahamnen,174,"2,225",13,Östermalm,Uppland
6,Jakob,62,201,3,Norrmalm,Uppland
7,Klara,71,"1,597",22,Norrmalm,Uppland
8,Kristineberg,63,"5,572",88,Kungsholmen,Uppland
9,Kungsholm,107,"18,465",173,Kungsholmen,Uppland


In [6]:
address = 'Stockholm, Sweden'
geolocator = Nominatim(user_agent="sto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Stockholm, Sweden are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Stockholm, Sweden are 59.3251172, 18.0710935.


In [7]:
def get_latilong(District):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Stockholm, Sweden'.format(District))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('Gärdet')

[59.33361000000008, 18.113360000000057]

In [8]:
# Retrieve Districts' Coordinates
District = df['District']    
coords = [ get_latilong(District) for District in District.tolist() ]

# Add columns Latitude & Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [9]:
df[df.District == 'Fredhäll']

,District,Area,Population,Density,Borough,Province,Latitude,Longitude
1,Fredhäll,38,"4,958",130,Kungsholmen,Uppland,59.33103,18.00545


In [10]:
df.head(15)

,District,Area,Population,Density,Borough,Province,Latitude,Longitude
0,Djurgården,290,788,3,Östermalm,Uppland,59.324620,18.097800
1,Fredhäll,38,"4,958",130,Kungsholmen,Uppland,59.331030,18.005450
2,Gustav Vasa,80,"12,911",161,Norrmalm,Uppland,59.342500,18.047750
3,Gärdet,459,"18,158",40,Östermalm,Uppland,59.333610,18.113360
4,Hedvig Eleonora,58,"10,387",179,Östermalm,Uppland,59.335240,18.080460
5,Hjorthagen-Värtahamnen,174,"2,225",13,Östermalm,Uppland,59.355180,18.100200
6,Jakob,62,201,3,Norrmalm,Uppland,59.329270,18.066005
7,Klara,71,"1,597",22,Norrmalm,Uppland,59.334670,18.068222
8,Kristineberg,63,"5,572",88,Kungsholmen,Uppland,59.336620,18.004860
9,Kungsholm,107,"18,465",173,Kungsholmen,Uppland,59.329010,18.048540


In [11]:
# Save it, for future reference
df.to_csv('stockholm.csv',index=False)

In [12]:
map_stockholm = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, district, borough in zip(df['Latitude'], df['Longitude'], df['District'], df['Borough']):
    label = '{}, {}'.format(district, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_stockholm)
map_stockholm

## Methodology <a name="methodology"></a>

After successfully extracting the information about Stockholm city centre from its Wikipedia page, and visualizing it on the city map, we will now use the Foursquare API to explore venues around the area. 

In [13]:
CLIENT_ID = '****' # your Foursquare ID
CLIENT_SECRET = '****' # your Foursquare Secret
VERSION = '20201106'
LIMIT = 100 # A default Foursquare API limit value

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
stockholm_venues = getNearbyVenues(names=df['District'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Djurgården
Fredhäll
Gustav Vasa
Gärdet
Hedvig Eleonora
Hjorthagen-Värtahamnen
Jakob
Klara
Kristineberg
Kungsholm
Lilla Essingen
Mariatorget
Marieberg
Mellersta Högalid
Norra Adolf Fredrik
Norra Johannes
Norra Högalid
Norra Sofia
Oscars Kyrka
Reimersholme-Långholmen
Roslagstull
Stadshagen
Stora Essingen
Storkyrkan
Stureplan-Lärkstaden
Södra Adolf Fredrik
Södra Hammarbyhamnen
Södra Högalid
Södra Johannes
Södra Sofia
Södra Station
Tekniska Högskolan
Universitetet
Västra Katarina
Västra Matteus
Östra Katarina
Östra Matteus
Östra Sankt Göran
Total


In [16]:
print(stockholm_venues.shape)
stockholm_venues.head()

(1915, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Djurgården,59.32462,18.0978,Skansen,59.325313,18.101222,Museum
1,Djurgården,59.32462,18.0978,Liljevalchs Konsthall,59.325767,18.096091,Art Gallery
2,Djurgården,59.32462,18.0978,Gröna Lund,59.323895,18.096267,Theme Park
3,Djurgården,59.32462,18.0978,ABBA The Museum / Swedish Music Hall of Fame,59.325007,18.096971,Museum
4,Djurgården,59.32462,18.0978,Restauration Gubbhyllan,59.324839,18.100985,Scandinavian Restaurant


In [17]:
stockholm_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Djurgården,57,57,57,57,57,57
Fredhäll,8,8,8,8,8,8
Gustav Vasa,94,94,94,94,94,94
Gärdet,6,6,6,6,6,6
Hedvig Eleonora,100,100,100,100,100,100
Hjorthagen-Värtahamnen,14,14,14,14,14,14
Jakob,57,57,57,57,57,57
Klara,100,100,100,100,100,100
Kristineberg,42,42,42,42,42,42


In [18]:
print('There are {} uniques categories.'.format(len(stockholm_venues['Venue Category'].unique())))

There are 206 uniques categories.


In [19]:
# one hot encoding
stockholm_onehot = pd.get_dummies(stockholm_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
stockholm_onehot['Neighborhood'] = stockholm_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [stockholm_onehot.columns[-1]] + list(stockholm_onehot.columns[:-1])
stockholm_onehot = stockholm_onehot[fixed_columns]

stockholm_onehot.head()

,Neighborhood,ATM,American Restaurant,Amphitheater,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bar,Beach,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Camera Store,Canal,Candy Store,Caribbean Restaurant,Castle,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Church,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Dive Bar,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Venue,National Park,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Palace,Park,Pedestrian Plaza,Persian Restaurant,Peruvian Restaurant,Pet Store,Photography Lab,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Pub,Rafting,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Road,Roof Deck,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Snack Place,Soccer Field,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Track,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Djurgården,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Djurgården,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Djurgården,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [20]:
stockholm_onehot.shape

(1915, 207)

In [21]:
stockholm_grouped = stockholm_onehot.groupby('Neighborhood').mean().reset_index()
stockholm_grouped

,Neighborhood,ATM,American Restaurant,Amphitheater,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bar,Beach,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Camera Store,Canal,Candy Store,Caribbean Restaurant,Castle,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Church,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Dive Bar,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Venue,National Park,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Palace,Park,Pedestrian Plaza,Persian Restaurant,Peruvian Restaurant,Pet Store,Photography Lab,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Pub,Rafting,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Road,Roof Deck,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Snack Place,Soccer Field,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Track,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Djurgården,0.000000,0.000000,0.000000,0.035088,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017544,0.000000,0.000000,0.000000,0.035088,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035088,0.000000,0.000000,0.000000,0.00000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035088,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017544,0.000000,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.035088,0.000000,0.000000,0.035088,0.000000,0.000000,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.00000,0.017544,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0

In [22]:
stockholm_grouped.shape

(38, 207)

In [23]:
num_top_venues = 5

for hood in stockholm_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = stockholm_grouped[stockholm_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Djurgården----
                          venue  freq
0  Theme Park Ride / Attraction  0.21
1       Scandinavian Restaurant  0.16
2                        Museum  0.07
3                  Tram Station  0.05
4                   Music Venue  0.05


----Fredhäll----
               venue  freq
0               Café  0.38
1               Park  0.12
2  Indian Restaurant  0.12
3         Playground  0.12
4          Mini Golf  0.12


----Gustav Vasa----
                     venue  freq
0                     Café  0.09
1              Pizza Place  0.07
2                   Bakery  0.05
3       Italian Restaurant  0.04
4  Scandinavian Restaurant  0.03


----Gärdet----
                       venue  freq
0             Science Museum  0.33
1            Harbor / Marina  0.17
2             History Museum  0.17
3  Middle Eastern Restaurant  0.17
4                     Museum  0.17


----Hedvig Eleonora----
                     venue  freq
0  Scandinavian Restaurant  0.14
1                    Hotel  0.06


## Most popular venues around the neighborhood

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = stockholm_grouped['Neighborhood']

for ind in np.arange(stockholm_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(stockholm_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Djurgården,Theme Park Ride / Attraction,Scandinavian Restaurant,Museum,Tram Station,Music Venue
1,Fredhäll,Café,Indian Restaurant,Thai Restaurant,Park,Mini Golf
2,Gustav Vasa,Café,Pizza Place,Bakery,Italian Restaurant,Indian Restaurant
3,Gärdet,Science Museum,Middle Eastern Restaurant,History Museum,Harbor / Marina,Museum
4,Hedvig Eleonora,Scandinavian Restaurant,Hotel,Italian Restaurant,Seafood Restaurant,Clothing Store


In [26]:
stockholm_merged = df
stockholm_merged = stockholm_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='District')
stockholm_merged = stockholm_merged.rename({'District': 'Neighborhood'}, axis=1)

In [27]:
stockholm_merged.head()

,Neighborhood,Area,Population,Density,Borough,Province,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Djurgården,290,788,3,Östermalm,Uppland,59.32462,18.09780,Theme Park Ride / Attraction,Scandinavian Restaurant,Museum,Tram Station,Music Venue
1,Fredhäll,38,"4,958",130,Kungsholmen,Uppland,59.33103,18.00545,Café,Indian Restaurant,Thai Restaurant,Park,Mini Golf
2,Gustav Vasa,80,"12,911",161,Norrmalm,Uppland,59.34250,18.04775,Café,Pizza Place,Bakery,Italian Restaurant,Indian Restaurant
3,Gärdet,459,"18,158",40,Östermalm,Uppland,59.33361,18.11336,Science Museum,Middle Eastern Restaurant,History Museum,Harbor / Marina,Museum
4,Hedvig Eleonora,58,"10,387",179,Östermalm,Uppland,59.33524,18.08046,Scandinavian Restaurant,Hotel,Italian Restaurant,Seafood Restaurant,Clothing Store


In [28]:
stockholm_merged = stockholm_merged.dropna()

In [29]:
stockholm_merged2 = stockholm_merged.drop(['Area', 'Population', 'Borough', 'Province', '4th Most Common Venue', '5th Most Common Venue'], axis=1)

In [30]:
stockholm_merged2.head()

,Neighborhood,Density,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Djurgården,3,59.32462,18.09780,Theme Park Ride / Attraction,Scandinavian Restaurant,Museum
1,Fredhäll,130,59.33103,18.00545,Café,Indian Restaurant,Thai Restaurant
2,Gustav Vasa,161,59.34250,18.04775,Café,Pizza Place,Bakery
3,Gärdet,40,59.33361,18.11336,Science Museum,Middle Eastern Restaurant,History Museum
4,Hedvig Eleonora,179,59.33524,18.08046,Scandinavian Restaurant,Hotel,Italian Restaurant


## Clustering the Neighborhoods using K-means

In [31]:
# Using K-Means to cluster neighborhood into 5 clusters
stockholm_grouped_clustering = stockholm_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=5, random_state=0).fit(stockholm_grouped_clustering)
kmeans.labels_[0:10]

array([2, 3, 0, 1, 2, 0, 2, 2, 4, 0], dtype=int32)

In [32]:
stockholm_merged2.insert(4, 'Cluster Labels', kmeans.labels_)

In [33]:
stockholm_merged2['Cluster Labels'] = stockholm_merged2['Cluster Labels'].astype(int)
stockholm_merged2.head()

,Neighborhood,Density,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Djurgården,3,59.32462,18.09780,2,Theme Park Ride / Attraction,Scandinavian Restaurant,Museum
1,Fredhäll,130,59.33103,18.00545,3,Café,Indian Restaurant,Thai Restaurant
2,Gustav Vasa,161,59.34250,18.04775,0,Café,Pizza Place,Bakery
3,Gärdet,40,59.33361,18.11336,1,Science Museum,Middle Eastern Restaurant,History Museum
4,Hedvig Eleonora,179,59.33524,18.08046,2,Scandinavian Restaurant,Hotel,Italian Restaurant


## Visualize the cluster!

In [34]:
kclusters = 5

In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(stockholm_merged2['Latitude'], 
                                  stockholm_merged2['Longitude'], 
                                  stockholm_merged2['Neighborhood'], 
                                  stockholm_merged2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Results and Discussion <a name="results"></a>

As a result, the neighborhoods around the Stockholm city centre were segmented into 5 clusters using the K-means clustering approach (unsupervised machine learning), where each cluster represent a group of similar neighborhods. In the following data frames, one can observe the details about the venues in each of the 5 clusters. 

**First cluster** (see Cluster Label 0) represents the neighborhoods sharing similar venues, such as café, pub, pizza place, restaurants, gym etc. being the most popular. This is a nice neighborhood to visit if you want to enjoy a nice variety of food, that includes Scandanavian, Thai, and Italian choices, to mention a few. This neighborhood is densely populated and quite popular amongst the locals and the visitors. It is also well connected with public transport, making it easier to commute around the area. Also, there are hotels nearby, making it a perfect neighborhood for tourists to stay and explore. Other facilities include supermarket, clothing store, bookstore and shopping plaza.

**Second cluster** (see Cluster Label 1) consists of a less crowded neighborhood "Gärdet", which is popular for Science and History museums and, authentic middle eastern restaurant. This is a nice choice of neighborhood if you enjoy visiting museums, and prefer less crowded places.

**Third cluster** (see Cluster Label 2) is dominated by neighborhoods with tourist attractions such as theme parks, skate park, beaches, lake, museums, and is also popular for the nightlife. There are a variety of restaurants in this neighborhood, including Sushi restaurant. Also, there are hotels and hostels in the vicinity, and the neighborhood is well connected with the public transport.

**Fourth cluster** (see Cluster Label 3) consists of 2 areas, popular for café, restaurants, coffee shop and bookstore. This area is also famous for an Indian restaurant.

**Fifth cluster** (see Cluster Label 4) consists of neighborhoods popular for restaurants, parks, bar, grocery store, etc. and public transport is available.  

In [36]:
stockholm_merged2.loc[stockholm_merged2['Cluster Labels'] == 0, stockholm_merged2.columns[[0]+[1]+list(range(5, stockholm_merged2.shape[1]))]]

,Neighborhood,Density,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
2,Gustav Vasa,161,Café,Pizza Place,Bakery
5,Hjorthagen-Värtahamnen,13,Light Rail Station,Pizza Place,Thrift / Vintage Store
9,Kungsholm,173,Café,Scandinavian Restaurant,Italian Restaurant
11,Mariatorget,227,Pub,Bar,Coffee Shop
14,Norra Adolf Fredrik,201,Café,Hotel,Scandinavian Restaurant
17,Norra Sofia,133,Café,Scandinavian Restaurant,Clothing Store
21,Stadshagen,36,Gym / Fitness Center,Scandinavian Restaurant,Supermarket
22,Stora Essingen,55,Plaza,Asian Restaurant,Bus Station
25,Södra Adolf Fredrik,106,Café,Hotel,Scandinavian Restaurant
26,Södra Hammarbyhamnen,85,Pier,Bus Stop,Café


In [37]:
stockholm_merged2.loc[stockholm_merged2['Cluster Labels'] == 1, stockholm_merged2.columns[[0]+[1]+list(range(5, stockholm_merged2.shape[1]))]]

,Neighborhood,Density,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
3,Gärdet,40,Science Museum,Middle Eastern Restaurant,History Museum


In [38]:
stockholm_merged2.loc[stockholm_merged2['Cluster Labels'] == 2, stockholm_merged2.columns[[0]+[1]+list(range(5, stockholm_merged2.shape[1]))]]

,Neighborhood,Density,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Djurgården,3,Theme Park Ride / Attraction,Scandinavian Restaurant,Museum
4,Hedvig Eleonora,179,Scandinavian Restaurant,Hotel,Italian Restaurant
6,Jakob,3,Hotel,Plaza,Café
7,Klara,22,Scandinavian Restaurant,Hotel,Burger Joint
12,Marieberg,41,Park,Scandinavian Restaurant,Skate Park
16,Norra Högalid,203,Bar,Bakery,Sushi Restaurant
18,Oscars Kyrka,132,Scandinavian Restaurant,Park,Italian Restaurant
19,Reimersholme-Långholmen,45,Hotel,Beach,Other Nightlife
20,Roslagstull,301,Sushi Restaurant,Miscellaneous Shop,Lake
23,Storkyrkan,72,Scandinavian Restaurant,Plaza,Café


In [39]:
stockholm_merged2.loc[stockholm_merged2['Cluster Labels'] == 3, stockholm_merged2.columns[[0]+[1]+list(range(5, stockholm_merged2.shape[1]))]]

,Neighborhood,Density,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
1,Fredhäll,130,Café,Indian Restaurant,Thai Restaurant
33,Västra Katarina,156,Café,Coffee Shop,Bookstore


In [40]:
stockholm_merged2.loc[stockholm_merged2['Cluster Labels'] == 4, stockholm_merged2.columns[[0]+[1]+list(range(5, stockholm_merged2.shape[1]))]]

,Neighborhood,Density,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
8,Kristineberg,88,Park,Middle Eastern Restaurant,Bus Station
10,Lilla Essingen,196,Grocery Store,Restaurant,Thai Restaurant
13,Mellersta Högalid,184,Bar,Bakery,Sushi Restaurant
15,Norra Johannes,137,Scandinavian Restaurant,Café,Hostel
27,Södra Högalid,58,Bar,Bakery,Sushi Restaurant
31,Tekniska Högskolan,22,Sushi Restaurant,Italian Restaurant,Gym / Fitness Center


## Conclusion <a name="conclusion"></a>

The purpose of this project was to explore the Stockholm city centre and find out the most popular places to visit.
Using the Foursquare API, we gathered the location data about the popular venues around the Stockholm city centre. Using unsupervised machine learning approach, K-means clustering, we clustered the locations to generate the major areas of interest containing the most popular places to explore. 

Final decision on the best neighborhood to visit will be made by the tourists, visitors or investors, based on their personal preferences about the specific characteristics they like in a neighborhood in every recommended zone (or cluster).

For example: 

* If you are a tourist visiting the Stockholm city center, the neighborhoods in the third cluster are a must visit! The venues suggested in the first cluster are also very interesting. 

* If you are an investor, interested in the restaurant business,  do check the neighborhoods and venues in the first, third and fifth cluster.

* If you want to visit museums, check out the first and third cluster.

* Do check out the map above to explore the neighborhoods, and refer to the tables for information about the venues that interests you.

Thank you for stopping by! <br>
This page will be further updated, as I am still exploring the power of data science! <br>